In [1]:
#### relative data, where the replicates were considered, their OD relative ratio taken and the mean evaluated
#### modification : remove wells that correspond to high variability and bistability (three such wells)

#### generate permanent data for test-train split

In [3]:
import pandas as pd
import random
import time
import numpy as np


from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import KFold



from sklearn.ensemble import RandomForestRegressor

import warnings
warnings.filterwarnings("ignore")

B = pd.read_pickle("~/bge-analysis-simv3/B.pkl")
A = pd.read_pickle("~/bge-analysis-simv3/A.pkl")


pa_redv1 = pd.read_pickle("~/compressed_sensingv1/realdatasets/misc/ophelli-ryan/eff-16-list1-red-pa.pkl")

##### compile the well numbers for each species for its presence 

species_ones = []
for i in range(16):
    species_ones.append(pa_redv1[pa_redv1[i]==1.0]["well number"].values)

ones_len = [] 
for ii in range(len(species_ones)):
    ones_len.append(len(species_ones[ii])) 

sst_redv1 = pd.read_pickle("~/compressed_sensingv1/realdatasets/OV-16sp-FUNCTIONS/16-list1-allmetabols.pkl")
sst_redv1.head(4)

,2,3,5,6,7,9,10,11,15,16,...,18,19,20,21,22,well index,butyrate,acetate,lactate,succinate
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.000000,0.977078,0.0,1.0,19.891379,3.802050,27.542614,2.614571
128,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.520201,0.0,...,0.0,0.0,0.000000,0.000000,0.0,8.0,0.383275,2.611108,26.853592,2.116042
130,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.284509,0.0,...,0.0,0.0,0.000000,0.640232,0.0,9.0,14.815900,4.867583,25.578500,3.605217
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.995363,0.000000,0.0,16.0,19.907547,5.276729,45.619753,2.863171


In [6]:
metabolite_list = ["butyrate","lactate","acetate","succinate"]
random_state = None
cv_splitsl = [3,5,7,10]

In [7]:
sst_redv1["succinate"]

2       2.614571
128     2.116042
130     3.605217
4       2.863171
6       2.982667
          ...   
1440    1.215200
289     0.000000
417     0.000000
2465    0.000000
368     0.000000
Name: succinate, Length: 187, dtype: float64

In [16]:
pa_redv1[]

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,well number
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,2
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,128
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,9.0,130
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,16.0,4
10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,17.0,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
457,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,25165896.0,1440
460,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,25167872.0,289
461,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,25167880.0,417
462,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,25176072.0,2465


In [58]:
dtr = []
dte = [] 
for cv_splits in cv_splitsl: 
    for metabols in metabolite_list:
        abunvec = sst_redv1[metabols]
        #abunvec.index = np.arange(0,2**7,1)
        listl = abunvec.index 
        data_present = list(abunvec.index)
        random.shuffle(data_present)
        data_present1 = np.array(data_present)
        kf = KFold(n_splits=cv_splits)
        kf.get_n_splits(data_present1)
        KFold(n_splits=cv_splits, random_state=93, shuffle=True)
        count = 0 
        for train_index, test_index in kf.split(data_present1):
            X_train, X_test = data_present1[train_index], data_present1[test_index]
            rinew = list(X_train)
            rileft = list(X_test)
            #y3 = abunvec.T[rinew].T[0].values  
            dtr.append([cv_splits,count,rinew,metabols])
            dte.append([cv_splits,count,rileft,metabols])
            count = count + 1


In [59]:
dtrf = pd.DataFradtrf = pd.DataFrame(dtr,columns=["k-fold","fold number","train data","metabols"])

dtef = pd.DataFrame(dte,columns=["k-fold","fold number","test data","metabols"])

dtrf.to_pickle("traindataRELfried4.pkl")
dtef.to_pickle("testdataRELfried4.pkl")


In [60]:
def flatten(l):
    return [item for sublist in l for item in sublist]

In [61]:
dtef

,k-fold,fold number,test data,metabols
0,3,0,"[36870, 48, 514, 8355, 1156, 40965, 4231, 211,...",butyrate
1,3,1,"[164, 36996, 3, 2049, 45, 167, 242, 34, 135, 2...",butyrate
2,3,2,"[25121, 16388, 549, 36994, 8193, 65, 41093, 22...",butyrate
3,3,0,"[258, 36870, 54, 37024, 242, 128, 2052, 2177, ...",lactate
4,3,1,"[192, 47, 2182, 2465, 18, 3, 2049, 10, 4128, 3...",lactate
...,...,...,...,...
95,10,5,"[16, 210, 2177, 9, 8355, 37, 8357, 4097, 1188,...",succinate
96,10,6,"[16416, 2050, 128, 416, 174, 2084, 8196, 514, ...",succinate
97,10,7,"[4231, 52, 160, 4096, 641, 4224, 8320, 16545, ...",succinate
98,10,8,"[32768, 36996, 3075, 32, 2048, 134, 4, 1156, 2...",succinate
